In [ ]:
from kubernetes import client, config
import os

# Load Kubernetes configuration (assumes running in cluster or ~/.kube/config is set up)
config.load_kube_config()  # Use this for local development, else use config.load_incluster_config()

# Define the path to the script
script_file = "gpt-j-6b.py"
requirements_file = "requirements.txt"

# Check if the file exists and read its content
if os.path.exists(script_file):
    with open(script_file, 'r') as file:
        script_content = file.read()
else:
    raise FileNotFoundError(f"{script_file} not found")

if os.path.exists(requirements_file):
    with open(requirements_file, 'r') as file:
        requirements_content = file.read()
else:
    raise FileNotFoundError(f"{requirements_file} not found")

# Create a ConfigMap metadata
metadata = client.V1ObjectMeta(
    name="script-config-map",
    namespace="mark-dsp"  # Specify your namespace if different from default
)

# Create the ConfigMap with the file content as data
config_map = client.V1ConfigMap(
    api_version="v1",
    kind="ConfigMap",
    metadata=metadata,
    data={  # Data in the ConfigMap (key-value pair)
        "gpt-j-6b.py": script_content,
        "requirements.txt": requirements_content,
    }
)

# Initialize API instance
v1 = client.CoreV1Api()

# Create the ConfigMap in the cluster
try:
    response = v1.create_namespaced_config_map(namespace="mark-dsp", body=config_map)
    print(f"ConfigMap {response.metadata.name} created successfully")
except client.exceptions.ApiException as e:
    print(f"Exception when creating ConfigMap: {e}")

In [ ]:
import os
from kubernetes import client, config
import base64
config.load_kube_config()

# Define the name of the secret and the namespace
secret_name = "my-secret"
namespace = "mark-dsp"

# Create a dictionary to store environment variables as secret data
secret_data = {
    'AWS_ACCESS_KEY_ID': os.environ.get('AWS_ACCESS_KEY_ID'),
    'AWS_SECRET_ACCESS_KEY': os.environ.get('AWS_SECRET_ACCESS_KEY'),
    'AWS_BUCKET_URI': os.environ.get('AWS_BUCKET_URI'),
    'AWS_DEFAULT_REGION': os.environ.get('AWS_DEFAULT_REGION')
}

# Encode the values in base64 as Kubernetes requires it
encoded_data = {k: base64.b64encode(v.encode('utf-8')).decode('utf-8') for k, v in secret_data.items()}



# Define the secret metadata and type
secret = client.V1Secret(
    api_version="v1",
    kind="Secret",
    metadata=client.V1ObjectMeta(name=secret_name, namespace=namespace),
    data=encoded_data,
    type="Opaque"
)

# Create the API client and use it to create the secret
api_instance = client.CoreV1Api()

try:
    api_instance.create_namespaced_secret(namespace=namespace, body=secret)
    print(f"Secret '{secret_name}' created successfully in namespace '{namespace}'")
except client.exceptions.ApiException as e:
    if e.status == 409:
        print(f"Secret '{secret_name}' already exists in namespace '{namespace}'.")
    else:
        print(f"Failed to create secret: {e}")

In [ ]:
from kfp import dsl
from kfp import compiler
from kfp import kubernetes

@dsl.component(packages_to_install=["codeflare-sdk"], base_image="registry.access.redhat.com/ubi9/python-39:latest")
def hackathon_fn() -> int:
    from codeflare_sdk.cluster.auth import TokenAuthentication
    from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
    import os
    import time
    import shutil

    # Source path (file you want to copy)
    source_1 = '/mnt/config/gpt-j-6b.py'
    source_2 = '/mnt/config/requirements.txt'
    
    current_dir = os.getcwd()
    # Destination path (where you want to copy the file)
    destination_1 = f'{current_dir}/gpt-j-6b.py' 
    destination_2 = f'{current_dir}/requirements.txt'
    

    # Copy the file from source to destination
    shutil.copy(source_1, destination_1)
    print(f"File copied from {source_1} to {destination_1}")
    shutil.copy(source_2, destination_2)
    print(f"File copied from {source_2} to {destination_2}")

    
    num_cpu=10
    num_memory=50

    cluster = Cluster(ClusterConfiguration(
        name='raytest',
        head_extended_resource_requests={'nvidia.com/gpu':1}, # For GPU enabled workloads set the head_extended_resource_requests and worker_extended_resource_requests
        worker_extended_resource_requests={'nvidia.com/gpu':1},
        num_workers=3,
        head_cpus=num_cpu,
        head_memory=num_memory,
        worker_cpu_requests=num_cpu,
        worker_cpu_limits=num_cpu,
        worker_memory_requests=num_memory,
        worker_memory_limits=num_memory,
        write_to_file=False,
    ))

    # always clean the resources
    cluster.down()
    print(cluster.status())
    cluster.up()
    cluster.wait_ready()
    print(cluster.status())
    print(cluster.details())

    client = cluster.job_client
    submission_id = client.submit_job(
        entrypoint="python gpt-j-6b.py",
        runtime_env={
            "working_dir": "./","pip": "requirements.txt",
            "env_vars": {
                "AWS_BUCKET_URI": os.environ.get("AWS_BUCKET_URI"),
                "AWS_ACCESS_KEY_ID": os.environ.get("AWS_ACCESS_KEY_ID"),
                "AWS_SECRET_ACCESS_KEY": os.environ.get("AWS_SECRET_ACCESS_KEY"),
                "AWS_DEFAULT_REGION": os.environ.get("AWS_DEFAULT_REGION"),
            },
        },
    )
    
    while True:
        status = client.get_job_status(submission_id)
        print(status)
        if status == 'SUCCEEDED':
            cluster.down()
            break
        if status == 'FAILED':
            cluster.down()
            break
        else:
            time.sleep(360)

    return 0

@dsl.pipeline(
    name="Ray Cluster Pipeline GPT-J-6B",
    description="Pipeline to run a CodeFlare Ray cluster with a ConfigMap"
)
def ray_fn_pipeline():
    hackathon_task = hackathon_fn()
    kubernetes.use_config_map_as_volume(
                hackathon_task,
                config_map_name='script-config-map',
                mount_path='/mnt/config',
            )
    kubernetes.use_secret_as_env(hackathon_task,
                                 secret_name='my-secret',
                                 secret_key_to_env={
                                     'AWS_BUCKET_URI': 'AWS_BUCKET_URI',
                                     'AWS_ACCESS_KEY_ID': 'AWS_ACCESS_KEY_ID',
                                     'AWS_SECRET_ACCESS_KEY': 'AWS_SECRET_ACCESS_KEY',
                                     'AWS_DEFAULT_REGION': 'AWS_DEFAULT_REGION',
                                 })


if __name__ == '__main__':
    compiler.Compiler().compile(ray_fn_pipeline, 'hackathon_pipeline.yaml')